In [2]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-05-07 19:10:18.820888: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 19:10:18.863804: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 19:10:19.595459: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [4]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import swallowing_recognition

In [5]:
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [6]:
from swallowing_recognition import wavelet
from swallowing_recognition import audio
from swallowing_recognition import dataset
from swallowing_recognition import dence_net

In [8]:
directory_path = parent_dir / 'dataset'
   
train_voice_folder = directory_path / 'washino' / 'voice'
train_cough_folder = directory_path / 'washino' / 'cough'
train_swallowing_folder = directory_path / 'washino' / 'swallowing'   
train_neck_movement_folder = directory_path / 'washino' / 'neck_movement'
train_shoulder_movement_folder = directory_path / 'washino' / 'shoulder_movement'
train_arm_movement_folder = directory_path / 'washino' / 'arm_movement'   
train_hitting_folder = directory_path / 'washino' / 'hitting'   
train_mumble_folder = directory_path / 'washino' / 'mumble'   

test_voice_folder = directory_path / 'shibata' / 'voice'
test_cough_folder = directory_path / 'shibata' / 'cough'
test_swallowing_folder = directory_path / 'shibata' / 'swallowing'    
test_movement_folder = directory_path / 'shibata' / 'movement'    
test_hitting_folder = directory_path / 'shibata' / 'hitting'    

# 引数は(データ数，画像サイズ縦，画像サイズ横，色RGB，クラス数)
# train_data = dataset.DataSet(500, 224, 224, 3, 5)
# test_data = dataset.DataSet(70, 224, 224, 3, 5)

train_data = dataset.DataSet(2000, 224, 224, 3, 2)
test_data = dataset.DataSet(62, 224, 224, 3, 2)

# train_data.folder_to_dataset(train_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# train_data.folder_to_dataset(train_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# train_data.folder_to_dataset(train_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# train_data.folder_to_dataset(train_movement_folder, np.array([0, 1, 0, 0, 0]), 3)
# train_data.folder_to_dataset(train_hitting_folder, np.array([1, 0, 0, 0, 0]), 4)

train_data.folder_to_dataset(train_swallowing_folder, np.array(0), 0)

In [9]:
train_data.folder_to_dataset(train_voice_folder, np.array(1), 700)
train_data.folder_to_dataset(train_hitting_folder, np.array(1), 800)
train_data.folder_to_dataset(train_cough_folder, np.array(1), 900)
train_data.folder_to_dataset(train_neck_movement_folder, np.array(1), 1000)
train_data.folder_to_dataset(train_shoulder_movement_folder, np.array(1), 1100)
train_data.folder_to_dataset(train_arm_movement_folder, np.array(1), 1200)
# train_data.folder_to_dataset(train_mumble_folder, np.array(1), 1300)

In [10]:
names = ['haruki', 'ezumi', 'hikaru', 'ibuki', 'kanata', 'kazuki', 'kishimoto', 'kosasa', 'mocchi', 'nakaryo', 'ruku', 'shibata', 'soichiro', 'tsuji']
print(len(names))

14


In [9]:
exit_name = 'haruki'

if exit_name in names:
    index_to_pop = names.index(exit_name)
    names.pop(index_to_pop)  # インデックスを使用して要素を削除
    print(f"{exit_name} has been removed.")
    print(names)
else:
    print(f"{exit_name} not found in the list.")

haruki has been removed.
['ezumi', 'hikaru', 'ibuki', 'kanata', 'kazuki', 'kishimoto', 'kosasa', 'mocchi', 'nakaryo', 'ruku', 'shibata', 'soichiro', 'tsuji']


In [11]:
start_num = 1300
for name in names:
    prediction_swallowing_folder = directory_path / name / 'swallowing'
    prediction_voice_folder = directory_path / name / 'voice'
    prediction_cough_folder = directory_path / name / 'cough'
    prediction_hitting_folder = directory_path / name / 'hitting'    
    prediction_movement_folder = directory_path / name / 'movement' 
    
    train_data.folder_to_dataset(prediction_swallowing_folder, np.array(0), start_num)
    start_num += 10
    train_data.folder_to_dataset(prediction_voice_folder, np.array(1), start_num)
    start_num += 10
    train_data.folder_to_dataset(prediction_cough_folder, np.array(1), start_num)
    start_num += 10
    train_data.folder_to_dataset(prediction_hitting_folder, np.array(1), start_num)
    start_num += 10
    train_data.folder_to_dataset(prediction_movement_folder, np.array(1), start_num)
    start_num += 10
    print(start_num)

1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000


In [13]:
test_data.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data.folder_to_dataset(test_cough_folder, np.array(1), 14)
test_data.folder_to_dataset(test_voice_folder, np.array(1), 28)
test_data.folder_to_dataset(test_movement_folder, np.array(1), 42)
test_data.folder_to_dataset(test_hitting_folder, np.array(1), 52)

In [25]:
print(type(train_data.data))
print((train_data.data).shape)
print((train_data.labels).shape)

<class 'numpy.ndarray'>
(2000, 224, 224, 3)
(2000,)


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data.data, train_data.labels, test_size=0.2, random_state=42)

# 結果の形状を確認
print("Train data shape:", X_train.shape)  # トレーニングデータの形状
print("Test data shape:", X_test.shape)    # テストデータの形状
print("Train labels shape:", y_train.shape) # トレーニングデータのラベルの形状
print("Test labels shape:", y_test.shape)   # テストデータのラベルの形状

count_0 = np.count_nonzero(y_train == 0)
count_1 = np.count_nonzero(y_train == 1)

print("train number of 0s:", count_0)
print("train number of 1s:", count_1)

count_0 = np.count_nonzero(y_test == 0)
count_1 = np.count_nonzero(y_test == 1)

print("test number of 0s:", count_0)
print("test number of 1s:", count_1)

Train data shape: (1600, 224, 224, 3)
Test data shape: (400, 224, 224, 3)
Train labels shape: (1600,)
Test labels shape: (400,)
train number of 0s: 668
train number of 1s: 932
test number of 0s: 172
test number of 1s: 228


In [19]:
from sklearn.model_selection import KFold
import numpy as np

# 5分割交差検証の設定
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# データを保持するためのリスト
train_data_splits = []
test_data_splits = []

# データのインデックスを分割
for train_index, test_index in kf.split(train_data.data):
    X_train, X_test = train_data.data[train_index], train_data.data[test_index]
    y_train, y_test = train_data.labels[train_index], train_data.labels[test_index]

    # 各フォールドのデータをリストに保存
    train_data_splits.append((X_train, y_train))
    test_data_splits.append((X_test, y_test))

# 結果の使用例
for i, (train, test) in enumerate(zip(train_data_splits, test_data_splits)):
    X_train, y_train = train
    X_test, y_test = test
    print(f"Fold {i+1}")
    print("Train data shape:", X_train.shape)
    print("Test data shape:", X_test.shape)
    print("Train labels shape:", y_train.shape)
    print("Test labels shape:", y_test.shape)
    print("Train number of 0s:", np.count_nonzero(y_train == 0))
    print("Train number of 1s:", np.count_nonzero(y_train == 1))
    print("Test number of 0s:", np.count_nonzero(y_test == 0))
    print("Test number of 1s:", np.count_nonzero(y_test == 1))



Fold 1
Train data shape: (1600, 224, 224, 3)
Test data shape: (400, 224, 224, 3)
Train labels shape: (1600,)
Test labels shape: (400,)
Train number of 0s: 668
Train number of 1s: 932
Test number of 0s: 172
Test number of 1s: 228
Fold 2
Train data shape: (1600, 224, 224, 3)
Test data shape: (400, 224, 224, 3)
Train labels shape: (1600,)
Test labels shape: (400,)
Train number of 0s: 666
Train number of 1s: 934
Test number of 0s: 174
Test number of 1s: 226
Fold 3
Train data shape: (1600, 224, 224, 3)
Test data shape: (400, 224, 224, 3)
Train labels shape: (1600,)
Test labels shape: (400,)
Train number of 0s: 657
Train number of 1s: 943
Test number of 0s: 183
Test number of 1s: 217
Fold 4
Train data shape: (1600, 224, 224, 3)
Test data shape: (400, 224, 224, 3)
Train labels shape: (1600,)
Test labels shape: (400,)
Train number of 0s: 687
Train number of 1s: 913
Test number of 0s: 153
Test number of 1s: 247
Fold 5
Train data shape: (1600, 224, 224, 3)
Test data shape: (400, 224, 224, 3)
Tra

In [24]:
# model = dence_net.DanceNet(5)

model = dence_net.DanceNet(2)

early_stopping = EarlyStopping(
    monitor='val_loss',  # 監視する値
    patience=10,         # 性能が改善されないエポック数
    verbose=1,           # 進行状況のメッセージ表示
    mode='min',          # 「減少」を監視
    restore_best_weights=True  # 最良のモデルの重みを復元
)

model_checkpoint = ModelCheckpoint(
    filepath='20240513_binary_model_best_all.keras',  # 保存するファイルのパス
    monitor='accuracy',        # 監視する値
    save_best_only=True,       # 最良のモデルのみを保存
    verbose=1,                 # 進行状況のメッセージ表示
    mode='max'                 # 監視
)

# model_checkpoint = ModelCheckpoint(
#     filepath='except_' + exit_name + '_20240507_binary_model_best.keras',  # 保存するファイルのパス
#     monitor='accuracy',        # 監視する値
#     save_best_only=True,       # 最良のモデルのみを保存
#     verbose=1,                 # 進行状況のメッセージ表示
#     mode='max'                 # 監視
# )

# model_checkpoint = ModelCheckpoint(
#     filepath='20240213_binary_model_best.keras',  # 保存するファイルのパス
#     monitor = "accuracy",
#     save_best_only=True,       # 最良のモデルのみを保存
#     verbose=1,                 # 進行状況のメッセージ表示
#     mode='max'                 # 「減少」を監視
# )

# 引数は(トレーニングデータ，ラベル，epochs，batch_size)
# model.training(train_data.data, train_data.labels, 40, 32, model_checkpoint = model_checkpoint)
# model.evaluate(test_data.data, test_data.labels)
# model.save('except_' + exit_name + '20240507_binary_ep40_bs32_v1.keras')

Epoch 1/40
45/45 [==============================] - ETA: 0s - loss: 0.2930 - accuracy: 0.8875
Epoch 1: accuracy improved from -inf to 0.88750, saving model to 20240513_binary_model_best_4.keras
45/45 [==============================] - 55s 141ms/step - loss: 0.2930 - accuracy: 0.8875 - val_loss: 8.2861 - val_accuracy: 0.4688
Epoch 2/40
45/45 [==============================] - ETA: 0s - loss: 0.2185 - accuracy: 0.9174
Epoch 2: accuracy improved from 0.88750 to 0.91736, saving model to 20240513_binary_model_best_4.keras
45/45 [==============================] - 5s 112ms/step - loss: 0.2185 - accuracy: 0.9174 - val_loss: 1.4437 - val_accuracy: 0.8250
Epoch 3/40
45/45 [==============================] - ETA: 0s - loss: 0.1559 - accuracy: 0.9389
Epoch 3: accuracy improved from 0.91736 to 0.93889, saving model to 20240513_binary_model_best_4.keras
45/45 [==============================] - 5s 112ms/step - loss: 0.1559 - accuracy: 0.9389 - val_loss: 0.9490 - val_accuracy: 0.8562
Epoch 4/40
45/45 [

Epoch 29/40
45/45 [==============================] - ETA: 0s - loss: 0.0260 - accuracy: 0.9924
Epoch 29: accuracy did not improve from 0.99583
45/45 [==============================] - 4s 96ms/step - loss: 0.0260 - accuracy: 0.9924 - val_loss: 0.1276 - val_accuracy: 0.9812
Epoch 30/40
45/45 [==============================] - ETA: 0s - loss: 0.0135 - accuracy: 0.9951
Epoch 30: accuracy did not improve from 0.99583
45/45 [==============================] - 4s 97ms/step - loss: 0.0135 - accuracy: 0.9951 - val_loss: 0.1928 - val_accuracy: 0.9625
Epoch 31/40
45/45 [==============================] - ETA: 0s - loss: 0.0284 - accuracy: 0.9924
Epoch 31: accuracy did not improve from 0.99583
45/45 [==============================] - 4s 97ms/step - loss: 0.0284 - accuracy: 0.9924 - val_loss: 0.4243 - val_accuracy: 0.8562
Epoch 32/40
45/45 [==============================] - ETA: 0s - loss: 0.0147 - accuracy: 0.9944
Epoch 32: accuracy did not improve from 0.99583
45/45 [==============================]

In [ ]:
use_data = 4

X_train, y_train = train_data_splits[use_data] 
X_test, y_test = test_data_splits[use_data]

model.training(X_train, y_train, 40, 32, model_checkpoint = model_checkpoint)
model.evaluate(X_test, y_test)
model.save('20240507_binary_ep40_bs32_4_v1.keras')

In [26]:
model.training(train_data.data, train_data.labels, 40, 32, model_checkpoint = model_checkpoint)
model.save('20240507_binary_ep40_bs32_all_v1.keras')

Epoch 1/40
57/57 [==============================] - ETA: 0s - loss: 0.1184 - accuracy: 0.9622
Epoch 1: accuracy did not improve from 0.99861
57/57 [==============================] - 17s 177ms/step - loss: 0.1184 - accuracy: 0.9622 - val_loss: 0.8125 - val_accuracy: 0.8350
Epoch 2/40
57/57 [==============================] - ETA: 0s - loss: 0.0806 - accuracy: 0.9706
Epoch 2: accuracy did not improve from 0.99861
57/57 [==============================] - 6s 105ms/step - loss: 0.0806 - accuracy: 0.9706 - val_loss: 0.2251 - val_accuracy: 0.9100
Epoch 3/40
57/57 [==============================] - ETA: 0s - loss: 0.0639 - accuracy: 0.9778
Epoch 3: accuracy did not improve from 0.99861
57/57 [==============================] - 6s 103ms/step - loss: 0.0639 - accuracy: 0.9778 - val_loss: 0.1606 - val_accuracy: 0.9600
Epoch 4/40
57/57 [==============================] - ETA: 0s - loss: 0.0444 - accuracy: 0.9850
Epoch 4: accuracy did not improve from 0.99861
57/57 [==============================] - 6

Epoch 31/40
57/57 [==============================] - ETA: 0s - loss: 0.0400 - accuracy: 0.9867
Epoch 31: accuracy did not improve from 0.99889
57/57 [==============================] - 6s 104ms/step - loss: 0.0400 - accuracy: 0.9867 - val_loss: 0.2578 - val_accuracy: 0.9550
Epoch 32/40
57/57 [==============================] - ETA: 0s - loss: 0.0306 - accuracy: 0.9889
Epoch 32: accuracy did not improve from 0.99889
57/57 [==============================] - 6s 104ms/step - loss: 0.0306 - accuracy: 0.9889 - val_loss: 1.7208 - val_accuracy: 0.8300
Epoch 33/40
57/57 [==============================] - ETA: 0s - loss: 0.0106 - accuracy: 0.9967
Epoch 33: accuracy did not improve from 0.99889
57/57 [==============================] - 6s 105ms/step - loss: 0.0106 - accuracy: 0.9967 - val_loss: 0.2325 - val_accuracy: 0.9650
Epoch 34/40
57/57 [==============================] - ETA: 0s - loss: 0.0097 - accuracy: 0.9967
Epoch 34: accuracy did not improve from 0.99889
57/57 [============================

In [ ]:
# model.evaluate_print()
test_data2 = dataset.DataSet(42, 224, 224, 3, 2)
test_data2.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data2.folder_to_dataset(test_cough_folder, np.array(1), 14)
test_data2.folder_to_dataset(test_voice_folder, np.array(1), 28)

In [ ]:
model.evaluate(test_data2.data, test_data2.labels)